# CNN with Keras

**This includes Visualizing of layers before flattening, and k fold cross validation for the model evaluation**

Group Members: 
* Akila Pieris, 
* Imantha Ahangama, 
* Ishan

# 1 Importing the Libraries and Configurations

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from random import randrange
import cv2

In [ ]:
classes = {'Black-grass': 0,
         'Charlock': 1,
         'Cleavers': 2,
         'Common Chickweed': 3,
         'Common wheat': 4,
         'Fat Hen': 5,
         'Loose Silky-bent': 6,
         'Maize': 7,
         'Scentless Mayweed': 8,
         'Shepherds Purse': 9,
         'Small-flowered Cranesbill': 10,
         'Sugar beet': 11}
image_size = 256
target_size = (256, 256)
batch_size = 32

# 2 Creating a dataframe of filenames and classes with Training Data

## 2.1 Scanning the file paths

In [ ]:
training_img_list = pd.DataFrame(columns=['filename','class'])
pathToTrainData='/kaggle/input/plant-seedlings-classification/train'


for dirname, _, filenames in os.walk(pathToTrainData):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        class_label = dirname.split('/')[-1]
        training_img_list = training_img_list.append({'filename':path , 'class':class_label}, ignore_index = True)

## 2.2 Shuffling the training image path list

In [ ]:

training_img_list = sklearn.utils.shuffle(training_img_list)
training_img_list.head(10)

# 3 Visualizing Training Data Sample

## 3.1 Loading All training data to a image generator

In [ ]:
idg_sample = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale=1./255,
                    horizontal_flip=True,
                    vertical_flip=True
                    )
sample_data = idg_sample.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                    target_size=target_size,
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=32,
                                                    shuffle=True,
                                                    seed=1,
                                                    classes = classes
                                                )

## 3.2 Defining a function to display unique set of images out of a image generator

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

def show_unique_grid(image_gen, figsize=(10,10)):
    dist_classes = [image_gen.classes.tolist().index(x) for x in set(image_gen.classes.tolist())]
    title_list = [image_gen.classes.tolist()[i] for i in dist_classes]
    im_set = [image_gen[i//32][0][i%32] for i in dist_classes]
    fig = plt.figure(None, (10,10),frameon=False)
    grid = ImageGrid(fig, 111, 
                     nrows_ncols=(3, 4),  
                     axes_pad=0.2, 
                     share_all=True,
                     )
    for i in range(12):
        ax = grid[i]
        ax.imshow(im_set[i],cmap='Greys_r')
        ax.set_title(title_list[i])
        ax.axis('off')


## 3.3 Sample of Each Class in the Training Data

In [ ]:
show_unique_grid(sample_data)
sample_data.class_indices

## 3.4 Class Distribution

In [ ]:
unique, counts = np.unique(sample_data.classes, return_counts=True)
dict1 = dict(zip(sample_data.class_indices, counts))

keys = dict1.keys()
values = dict1.values()

plt.xticks(rotation='vertical')
bar = plt.bar(keys, values)

# 4 Generating the CNN Model

## 4.1 Defining Model

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
import tensorflow.keras.optimizers as Optimizer
def get_model():
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(image_size, image_size, 3))
    vgg.trainable = False
    set_trainable = False
    for layer in vgg.layers:
        if layer in vgg.layers[-5:]:
            set_trainable = True
            
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    
    model = vgg.output
    model = tf.keras.layers.GlobalMaxPool2D()(model)
    model = tf.keras.layers.Flatten()(model) 
    model = tf.keras.layers.Dense(256, activation='relu')(model)
    model = tf.keras.layers.Dropout(0.2)(model)
    model = tf.keras.layers.Dense(64, activation='relu')(model)
    model = tf.keras.layers.BatchNormalization()(model)
    model = tf.keras.layers.Dense(12, activation='softmax')(model)
    
    model = Model(inputs=vgg.inputs, outputs= model)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## 4.2 Model Summary

In [ ]:
get_model().summary()

## 4.3 Visualizing Model

In [ ]:
from keras.utils import plot_model
plot_model(get_model(), show_shapes=True, show_layer_names=True)

## 4.4 Evaluating the Model doing a 5 fold cross validation

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


kfold = KFold(n_splits=5, shuffle=True)
fold = 1
accuracies = []
rand_num = randrange(100000)
for train, test in kfold.split(training_img_list):
    model = get_model()
    idg = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split=0.2
           )
    #Actual Training data for the model
    train_gen = idg.flow_from_dataframe(training_img_list.iloc[train,:],
                                        x_col="filename",
                                        y_col="class",
                                        subset='training',
                                        target_size=target_size,
                                        class_mode='categorical',
                                        batch_size=32,
                                        seed=1,
                                        shuffle=True,
                                        classes=classes
                                        )
    #Validation set for training the model, so the checkpoint can track the best model with max val accuracy
    val_gen   = idg.flow_from_dataframe(training_img_list.iloc[train,:],
                                        x_col="filename",
                                        y_col="class",
                                        subset='validation',
                                        target_size=target_size,
                                        class_mode='categorical',
                                        batch_size=32,
                                        seed=1,
                                        classes=classes
                                        )
    

    model_name = 'plant_classifier_'+str(rand_num)+'_fold_no_'+str(fold)+'.h5'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name,
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1
                                                    )
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', min_delta=0.0001)
    history = model.fit(train_gen,
                        epochs=5,
                        validation_data=val_gen,
                        callbacks = [checkpoint, es],
                        verbose = 1,
                        steps_per_epoch=5,
                        validation_steps=5
                        )
    # Testing for the fold
    idg2 = tf.keras.preprocessing.image.ImageDataGenerator(
                        rescale=1./255,
                        horizontal_flip=True,
                        vertical_flip=True
                       )
    test_gen = idg2.flow_from_dataframe(training_img_list.iloc[test,:],
                                        x_col="filename",
                                        y_col="class",
                                        target_size=target_size,
                                        class_mode='categorical',
                                        batch_size=32,
                                        seed=1,
                                        classes=classes,
                                        shuffle=False
                                        )
    model.load_weights('./'+model_name)
    predictions = model.predict(test_gen, steps=test_gen.samples)
    y_pred = []
    for i in range(0, predictions.shape[0]):
      y_class = predictions[i, :].argmax(axis=-1)
      y_pred.append(y_class)
    accuracies.append(accuracy_score(test_gen.classes, y_pred))
    print('Accuracy for the fold ',fold, ': ', accuracies[fold-1])
    fold += 1

## 4.5 Final 5-Cross Validation Score for the Model

In [ ]:
print('5 fold accuracies= ', accuracies)
print('5 fold accuracy = ', np.mean(accuracies))

# 5 Training the Model with full dataset

In [ ]:
idg3 = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            horizontal_flip=True,
            vertical_flip=True,
            validation_split=0.2
           )
train_gen = idg3.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                    target_size=target_size,
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=1,
                                                    classes=classes
                                                )
val_gen = idg3.flow_from_directory('../input/plant-seedlings-classification/train/',
                                                   target_size=(image_size, image_size),                                                   
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   seed=1,
                                                   classes=classes
                                                )

In [ ]:
model = get_model()
for layer in model.layers:
    print(layer.name+':',layer.trainable)

In [ ]:



model_name = 'plant_classifier_fully_trained_'+str(randrange(100000))+'.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name, #where to save the model
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose = 1)

history = model.fit(train_gen,
          epochs=20, # Increase number of epochs if you have sufficient hardware
          validation_data=val_gen,
          callbacks = [checkpoint],
          verbose = 1,
          steps_per_epoch=train_gen.n//train_gen.batch_size,
          validation_steps=val_gen.n//val_gen.batch_size
)

# 6 Visualizing the Output of the model before flattening


## 6.1 Getting the Model
Edit *layer_before_flattening* to have any layer before flattening to visualize the output from that layer

In [ ]:
from keras.models import Model
layer_before_flattening = 20 #Edit this with the layer you need the visualization for

viz_model = Model(inputs=model.inputs, outputs=model.layers[layer_before_flattening].output)
viz_model.summary()

## 6.2 Visualizing a Random Image on its original state.
Change the *image_no* to change the image(valid values: 0-31)

In [ ]:
image_no = 12

In [ ]:
plt.imshow(train_gen[0][0][image_no])

## 6.3 Visualizing the features from the output layer for above image

In [ ]:
nn = viz_model.predict(train_gen[0][0])

In [ ]:
grid = (8, 16) 
ix = 1
fig = plt.figure(figsize=(32,16))
for _ in range(grid[0]):
    for _ in range(grid[1]):
        # specify subplot and turn of axis
        ax = fig.add_subplot(grid[0], grid[1], ix)
        ax.set_xticks([])
        ax.set_yticks([])
        # plot filter channel in grayscale
        plt.imshow(nn[image_no, :, :, ix-1])
        ix += 1

plt.show()

# 7 Create Submission

## 7.1 Creating Data Generator for Test Data

In [ ]:
idg4 = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255,
            horizontal_flip=True,
            vertical_flip=True
           )
test_generator = idg4.flow_from_directory(
        directory='/kaggle/input/plant-seedlings-classification/',
        classes=['test'],
        target_size=target_size,
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical',
        seed=1)

## 7.2 Loading the model and Creating Submission

model_name has the previously created model for training. But can replace with any model

In [ ]:
final_model = get_model()
final_model.load_weights('./'+model_name)
predictions = final_model.predict(test_generator, steps=test_generator.samples)

class_list = []

for i in range(0, predictions.shape[0]):
  y_class = predictions[i, :].argmax(axis=-1)
  class_list += [list(classes.keys())[y_class]]

submission = pd.DataFrame()
submission['file'] = test_generator.filenames
submission['file'] = submission['file'].str.replace(r'test/', '')
submission['species'] = class_list

submission.to_csv('submission.csv', index=False)